# The Solow model with land

Imports and set magics:

In [192]:
import numpy as np
from scipy import optimize
import sympy as sm
from sympy import Symbol
from sympy.solvers import solve
sm.init_printing(use_unicode=True) # for pretty printing
from IPython.display import display
import matplotlib.pyplot as plt # baseline modul
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The **Solow model with land** contains:

* $K_t$ is capital
* $L_t$ is labor
* $A_t$ is technology
* $X$ is land
* $Y_t = F(K_t,A_tL_t,X)$ is GDP

**GDP** is defined as

$ Y_{t}=K_{t}^{\alpha}(A_{t}L_{t})^{\beta}X^{\kappa}$

**Saving** is a constant fraction of GDP

$ S_t = sY_t,\, s\in(0,1) $

such that **capital accumulates** according to

$ K_{t+1}=sY_t+(1-\delta)K_t $

**Labour and technology accoumulates** as

$ L_{t+1} = (1+n)L_t $

$ A_{t+1} = (1+g)A_t $

The **transition equations** then becomes

$ \tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[\tilde{k}_{t}^{\alpha}(\frac{X}{A_tL_t})^\kappa+(1-\delta)\tilde{k}_{t}] $

where $\tilde{k}_{t}= \dfrac{K_t}{A_tL_t}$


$ z_{t+1} = \frac{K_{t+1}}{Y_{t+1}} = \frac{K_{t+1}}{K_{t+1}^{\alpha}(A_{t+1}L_{t+1})^{\beta}X^{\kappa}} = \frac{K_{t+1}^{1-\alpha}}{(A_{t+1}L_{t+1})^{\beta}X^{\kappa}} \rightarrow z_{t+1} = \frac{1}{((1+g)(1+n))^\beta}(s+z_t(1-\delta))^{1-\alpha}z_t^\alpha$

# Steady state

## Analytical solution

We use **sympy** to find an analytical expression for the steady state, i.e. solve

$$ z^{\ast} = \frac{1}{((1+g)(1+n))^\beta}(s+z^{\ast}(1-\delta))^{1-\alpha}(z^{\ast})^\alpha $$

In [193]:
#We define all symbols
k = sm.symbols('k') #physical capital
X = sm.symbols('X') #land
z = sm.symbols('z') # #physical capital/GDP ratio
alpha = sm.symbols('alpha') #return on physical capital
beta = sm.symbols('beta') #labour force income share
kappa = sm.symbols('kappa') #return on land
delta = sm.symbols('delta') #depreciation rate on capital
s = sm.symbols('s') #savings/investment rate
g = sm.symbols('g') #growth in technology
n = sm.symbols('n') #growth in labour force

In [194]:
#Then we define the steady state equations

ss1 = sm.Eq(((s+z*(1-delta))**(1-alpha))*z**alpha/(((1+n)*(1+g))**beta), z)
ss1

In [195]:
sm.nonlinsolve([ss1], z)

⎧⎛⎧  │         ⎛      α                  -β                1 - α    ⎞⎫ ⎞⎫
⎨⎜⎨z │ z ∊ ℂ ∧ ⎝-z + z ⋅((g + 1)⋅(n + 1))  ⋅(s + z⋅(1 - δ))      = 0⎠⎬,⎟⎬
⎩⎝⎩  │                                                               ⎭ ⎠⎭

In [196]:
#KAN MÅSKE SLETTES, DA VI IKKE HAR TO LIGNINGER?
#We solve the steady state equations for z
sss1 = sm.solve((ss1), z, implicit = True)
sss1


When using $ 1 - \alpha = \beta + \kappa $, this can be reduced to:

$$ z_t = z = z^{\ast} = \frac{s}{((1+g)(1+n))^\frac{\beta}{\beta+\kappa}-(1-\delta)} $$

In [197]:
#We turn the solution into a Python funciton
ss_func = sm.lambdify((s,g,n,kappa,beta,delta),sss1)

## Numerical solution

We rewrite the equation for the steady state as

$ 0 = \frac{s}{((1+g)(1+n))^\frac{\beta}{\beta+\kappa}-(1-\delta)} -z^{\ast} $


now it is a root-finding problem.

In [198]:
#We define model parameters as plausible empirical values
s = 0.2
g = 0.02
n = 0.01
beta = 2/3
kappa = 1/6
delta = 0.06

In [199]:
#We solve numerically for the steady state:

def z_func(s,g,n,beta,kappa,delta):
    """args:
    s    (float): Investments
    g     (float): Growth in technology
    n     (float): Growth in labour force
    beta   (float): Labour force income share
    kappa   (float): Return on land
    delta (float): Depreciation rate 

    Returns: 
    The transition equation for z
    """
    result = lambda z: s/(((1+g)(1+n))**(beta/(beta+kappa))-(1-delta))
    return optimize.bisect(result, 1, 100, full_output=False)

print(f'z={z_func(s,g,n,kappa,beta,delta):.3f}')


TypeError: 'float' object is not callable

In [ ]:
#We solve numerically for the steady state:

def z_func(s,g,n,alpha,beta,delta):
    """args:
    s    (float): Investments
    g     (float): Growth in technology
    n     (float): Growth in labour force
    delta (float): Depreciation rate 
    alpha (float): Return on physical capital
    beta   (float): Return on technology*labour 

    Returns: 
    The transition equation for z
    """
    result = lambda z: s+z*(1-delta)**(1-alpha)*z**alpha/(((1+n)*(1+g))**beta)-z
    return optimize.bisect(result, 1, 100, full_output=False)

print(f'z*={z_func(s,g,n,alpha,beta,delta):.3f}')


In [ ]:
#We define the analytical solution
analSolution = ss_func(sh,sk,g,n,alpha,phi,delta)

#We define the numerical solution
numSolution = sol.x

print(f'Analytical solution is: k* = {analSolution[0]:.3f}, h* = {analSolution[1]:.3f}')
print(f'Numerical solution is: k* = {numSolution[1]:.3f}, h* = {numSolution[0]:.3f}')


As expected, the numerical and analytical steady state values are identical.

# Further analysis

## Phase diagrams

In [ ]:
#We define a function expressing the nullclines
def solve_ss(sh,sk,g,n,alpha,phi,delta):
    """args:
    sh    (float): Investments in human capital
    sk     float): Investments in physical capital
    g     (float): Growth in technology
    n     (float): Growth in labour force
    alpha (float): Return on physical capital
    phi   (float): Return on human capital
    delta (float): Depriciation rate
    
    Returns:
    Nullclines for physical and human capital
    """
    
    # a. grids
    k_vec = np.linspace(1e-4,3,500)
    h_vec_DeltaK0 = np.empty(500)
    h_vec_DeltaH0  = np.empty(500)
    
    # b. solve for each k in grid
    for i,k in enumerate(k_vec):
        #Solve for delta_h=0
        
        # i. objective
        obj = lambda h: -h_func(k,sh,sk,g,n,alpha,phi,delta,h)

        # ii. optimizer
        result = optimize.root_scalar(obj,method='brentq',bracket=[1e-20,50])
        
        h_vec_DeltaH0[i] = result.root

       
        #Solve for delta_k=0
        # i. objective
        obj = lambda h: -k_func(h,sh,sk,g,n,alpha,phi,delta,k)

        # ii. optimizer
        result = optimize.root_scalar(obj,method='brentq',bracket=[1e-20,50])

        h_vec_DeltaK0[i] = result.root

    return k_vec, h_vec_DeltaK0, h_vec_DeltaH0


#Solve the function
k_vec, h_vec_DeltaK0, h_vec_DeltaH0 = solve_ss(sh,sk,g,n,alpha,phi,delta)


#Create the plot
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(1,2,1)
ax.plot(k_vec,h_vec_DeltaK0, label=r'$\Delta \tilde{k}=0$', c='#005c2b')
ax.plot(k_vec,h_vec_DeltaH0, label=r'$\Delta \tilde{h}=0$', c='#61ba89')
ax.set_xlabel(r'$\tilde{k}$')
ax.set_ylabel(r'$\tilde{h}$')
ax.legend() 

#We mark the steady state
plt.scatter(sol.x[1],sol.x[0],color='black',s=80,zorder=2.5)

#Lines are drawn to mark ss-value on the axes
plt.axvline(sol.x[1],ymax=0.7,color='gray',linestyle='--') 
plt.axhline(sol.x[0],xmax=0.7,color='gray',linestyle='--') 

#Text is added to the plot
ax.text(1.15, 1.4, r'$\tilde{k}^*, \tilde{h}^*$', fontsize=12)

#The axis values are chosen
ax.set(xlim=(0, 2), ylim=(0, 2))

ax.set_title('Phase diagram');

Here the steady state occurs when the two nullclines intersect as illustrated above. The steady state values in the diagram are consistent with prior numerical and analytical results. 

We wish to see what happens, when the savings rate in human capital rises from its initial value

In [ ]:
#We solve the function with the new value sh' = 0.12
k_vec1,h_vec_DeltaK01, h_vec_DeltaH01 = solve_ss(0.12,sk,g,n,alpha,phi,delta)

#We define our analytical solution with the new value sh' = 0.12
analSolution2 = ss_func(0.12,sk,g,n,alpha,phi,delta)

#Create the plot
fig2 = plt.figure(figsize=(18,7))
ax = fig2.add_subplot(1,2,1)
ax.plot(k_vec,h_vec_DeltaK0, label=r'$\Delta \tilde{k}=0$', c='#005c2b')
ax.plot(k_vec,h_vec_DeltaH0, label=r'$\Delta \tilde{h}=0$', c='#61ba89')
ax.plot(k_vec1,h_vec_DeltaH01, label=r'$\Delta \tilde{h}´=0$', c='#a97ccc')
ax.set_xlabel(r'$\tilde{k}$')
ax.set_ylabel(r'$\tilde{h}$')
ax.legend()
#We mark the initial steady state
plt.scatter(sol.x[1],sol.x[0],color='black',s=80,zorder=2.5) 
#We mark the new steady state
plt.scatter(analSolution2[0],analSolution2[1],color='black',s=80,zorder=2.5)

#Lines are drawn to mark the initial ss-value on the axes
plt.axvline(sol.x[1],ymax=0.45,color='gray',linestyle='--') 
plt.axhline(sol.x[0],xmax=0.45,color='gray',linestyle='--')

#Lines are drawn to mark the new ss-value on the axes
plt.axvline(analSolution2[0],ymax=0.65,color='gray',linestyle='--')
plt.axhline(analSolution2[1],xmax=0.55,color='gray',linestyle='--')

#Text is added to the plot
ax.text(1.4, 2.0, r'$\tilde{k}^*, \tilde{h}^*$', fontsize=12)
ax.text(1.05, 1.4, r'$\tilde{k}^*´, \tilde{h}^*$´', fontsize=12)

#The axis values are chosen
ax.set(xlim=(0, 3), ylim=(0, 3)) 

ax.set_title('Phase diagram for $s_H$´ > $s_H$');

The new phase diagram shows how the steady state values of both $\tilde{h}$ and $\tilde{k}$ rises as a result of the higher savings rate. The rise in the physical capital per capita, $\tilde{k}$ is caused by the increase in the total savings of physical capital occuring as a result of the higher levels of $Y$ induced by the new, higher value of $H$.

## Golden rule

We know that $\tilde{y}_t^\ast=\tilde{k}_t^{\ast \alpha}\tilde{h}_t^{\ast \varphi}$

In [ ]:
#We define the y steady state function
def y_ss(sh,sk,g,n,alpha,phi,delta):
    """args:
    sh  (  float): Investments in human capital
    sk     float): Investments in physical capital
    g     (float): Growth in technology
    n     (float): Growth in labour force
    alpha (float): Return on physical capital
    phi   (float): return on human capital
    delta (float): depriciation rate
    
    Returns:
    Steady state value for total income
    """
    return ((sk**alpha*sh**(1-alpha))/(n+g+n*g+delta))**(alpha/(1-alpha-phi))*((sk**(1-phi)*sh**phi)/(n+g+n*g+delta))**(phi/(1-alpha-phi))

print(y_ss(sh,sk,g,n,alpha,phi,delta))

#We define the steady state function for c
def c_ss(sh,sk,g,n,alpha,phi,delta):
    """args:
    sh  (  float): Investments in human capital
    sk     float): Investments in physical capital
    g     (float): Growth in technology
    n     (float): Growth in labour force
    alpha (float): Return on physical capital
    phi   (float): return on human capital
    delta (float): depriciation rate
    
    Returns:
    Golden rule for private consumption
    """
    return (1-sk-sh)*((sk**alpha*sh**(1-alpha))/(n+g+n*g+delta))**(alpha/(1-alpha-phi))*((sk**(1-phi)*sh**phi)/(n+g+n*g+delta))**(phi/(1-alpha-phi)
                                                                                                                                )

In [ ]:
#We create a plot to show the variation in c when sh changes, all things equal

#we make an empty list to collect the differenc c-values
c_values = []
c_x = []

#We loop over different values of sh to find the corresponding values of c
for i in np.arange(0,0.9,0.001):
    c_values.append(c_ss(i,sk,g,n,alpha,phi,delta))
    c_x.append(i)

#Create the plot
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(1,2,1)
ax.plot(c_x,c_values, color='#a60550')
ax.set_xlabel(r'$s_H$')
ax.set_ylabel(r'$c^*$')

#We mark the maximum value of c and the corresponding value of sh
plt.axvline(0.45,ymax=0.95,color='gray',linestyle='--')
plt.axhline(max(c_values),xmax=1,color='gray',linestyle='--')
ax.set_title('$c^*$ for different $s_H$');

In [ ]:
#We find the largest ss-value of consumption pr. worker
print(f'The maximum private consumption per capita is: {max(c_values):.3f}')

#We find the sh value resulting in the maximum c value
print(f'The golden rule value of sh is: {(c_x[c_values.index(max(c_values))])}')

The optimal savings rate that maximises consumption is thus 0.45, which can be considered to be quite high. It seems that the rise in production caused by the increased savings outweighs the consumption lost due to a higher savings rate.

## Simulation

We want to simulate what happens when a permanent positive shock in $s_H$ occurs; when the savings of human capital rises permanently from a time period $t=2$ where $y_t=\dfrac{Y_t}{L_t}$

In [ ]:
#The initial values of our variables are defined
A0, K0, H0, L0, Y0 = 1, 1, 1, 1, 1


#We define the functions using variables for the first period
Lt = (1+n)*L0
Yt = K0**alpha*H0**phi*A0**(1-alpha-phi)

#An empty list is made
growthyt = []

for i in range(200):
    #A permanent shock to sh from period 2
    if i <= 1:
        sh = 0.1
    else:
        sh = 0.2
        
    #Y function in previous period
    Ytm1 = Yt
    
    #L function in previous period
    Ltm1 = Lt
    
    #y function in previous period
    ytm1 = 0
    ytm1 = Ytm1/Ltm1
    
    #K function in current period 
    Kt=sk*Ytm1+(1-delta)*K0
    K0=Kt
    
    #H function in current period
    Ht = sh*Ytm1+(1-delta)*H0
    H0=Ht
    
    #A function in current period
    At = (1+g)*A0
    A0=At
    
    #L function in current period
    Lt = (1+n)*L0
    L0=Lt
    
    #Y function in current period
    Yt=Kt**alpha*Ht**phi*At**(1-alpha-phi)
    
    #y function in current period
    yt = Yt/Lt
    
    #We calculate the growth in yt using log
    growthyt.append(np.log(yt)-np.log(ytm1))

#Create the plot
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(1,2,1)



#We discard the first growth rate since ytm1 doesn't have a well defined value for period 0
ax.plot(growthyt[1:], color='#076e62')

plt.axhline(growthyt[199],xmax=1,color='gray',linestyle='--')

ax.set_xlabel(r'time')
ax.set_ylabel(r'growth in $y_t$')
ax.set_title('Growth in $y_t$ with a permanent positive shock in $s_H$');

The graph shows how an increase in the savings rate of human capital causes a spike in the growth rate in production per worker. From that point on, the growth subsides and continues to do so until a stable growth path is achieved.

# Conclusion

Our calculations show that for our parameter values a steady state arises. In the basic Solow model the savings rate affects the steady state value positively. Likewise, the value of $s_H$ has a positive impact on the steady state values of $\tilde{h}^*$ and $\tilde{k}^*$, which is initially caused by a direct change in $\tilde{h}^*$ from which a spillover effect occurs, affecting both $\tilde{k}^*$ and production. Furthermore the consumption maximising value of $s_H$ is found to be $0.45$. The analysis also shows that different solving methods provides the same results in our case.